In [49]:
import pandas as pd
import numpy as np
import cv2
from keras.preprocessing import image
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.activations import elu
from sklearn.metrics import cohen_kappa_score
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.optimizers import Adam
from efficientnet.tfkeras import EfficientNetB5
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import logging

In [50]:
from __future__ import absolute_import, division, print_function, unicode_literals
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  8


#### Download arrays from data/npy_files that were created in '1. preprocessing'

In [ ]:
images = np.load("data/npy_files/images_arrays.npy", allow_pickle=True)
labels = np.load("data/npy_files/labels.npy", allow_pickle=True)
multi_labels = np.load("data/npy_files/multi_labels.npy", allow_pickle=True)

In [ ]:
df_labels = pd.read_csv('data/trainLabels_cropped.csv')
images_names = sorted(df_labels.image.values) 

In [ ]:
ys = df_labels.level.value_counts().values
xs = df_labels.level.value_counts().index
plt.rcParams["axes.labelsize"] = 12
fig = sns.barplot(x=xs, y=ys)
fig.set(xlabel='Severity level', ylabel='Number of observation', title="Distribution of classes")
plt.show()

#### We had an imbalance in class distribution and we deal with it in '1. prepricessing'

In [ ]:
def show_image(img):
    plt.imshow(img)
    
def preprocess_image_ben_grams(image, IMG_WIDTH, IMG_HEIGHT, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

def get_first_image_from_its_level(level=4):
    for ind_image, level_image in enumerate(labels):
        if level_image == level:
            return ind_image 

def get_each_class_image():
    each_class_image = []
    for level in [0, 1, 2, 3, 4]:
        each_class_image.append(get_first_image_from_its_level(level=level))
    return each_class_image

#### Before Ben Gram preprocessing

In [ ]:
each_class_images_inds = get_each_class_image()
plt.figure(figsize=(20,20)),
step = 0
for i in range(5):
    plt.subplot(330 + 1 + step)
    img_ind = each_class_images_inds[i]
    img = cv2.resize(images[img_ind], (324, 324))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  
    plt.title("severity: " + str(labels[img_ind]))
    step +=1

#### After Ben Gram preprocessing

In [ ]:
plt.figure(figsize=(20,20)),
for i in range(5):
    plt.subplot(330 + 1 + i)
    img_ind = each_class_images_inds[i]
    img = preprocess_image_ben_grams(images[img_ind], IMG_WIDTH=324, IMG_HEIGHT=324)
    show_image(img) 
    plt.title("severity: " + str(labels[img_ind]))

#### Create a callback for Kappa score. 


In [ ]:
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    
    :param model: A Keras model object
    :param generator: A Keras ImageDataGenerator object
    
    :return: A tuple with two Numpy Arrays. One containing the predictions
    and one containing the labels
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(len(generator.index_array)) / BATCH_SIZE)):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()

class Metrics(Callback):
    """
    A custom Keras callback for saving the best model
    according to the Quadratic Weighted Kappa (QWK) metric
    """
    def on_train_begin(self, logs={}):
        """
        Initialize list of QWK scores on validation data
        """
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Gets QWK score on the validation data
        
        :param epoch: The current epoch number
        """
        # Get predictions and convert to integers
        y_pred, labels = get_preds_and_labels(model, val_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)
        # We can use sklearns implementation of QWK straight out of the box
        # as long as we specify weights as 'quadratic'
        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        logging.info(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            logging.info("Validation Kappa has improved. Model saved!")
            self.model.save(MODEL_NAME)
        return

# Building and training part

In [ ]:
def create_datagen():
    return ImageDataGenerator(
        #zoom_range=0.15,  # set range for random zoom
        # set mode for filling points outside the input boundaries
        #fill_mode='constant',
        #cval=0.,  # value used for fill_mode = "constant"
        rotation_range=360,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        rescale=1 / 128,
        validation_split=0.2
    )

def data_generator(IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, prep_x_train, train_valid):
    if train_valid == "train":
        return create_datagen().flow(prep_x_train,
                                     y_train,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     subset="training",
                                     seed=2019)
    
    if train_valid == "valid":
        return create_datagen().flow(prep_x_train,
                                     y_train,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     subset="validation",
                                     seed=2019)

def build_model(transfer_learning_model, model_type):
    """
    transfer_learning_model - some pretrained architecture
    """
    model = Sequential()
    model.add(transfer_learning_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    
    if model_type == "effnet":
        model.add(layers.Dense(5, activation=elu))
        model.add(layers.Dense(1, activation="linear"))
        model.compile(loss='mse', 
                  metrics=['mse', 'acc'])
        
    if model_type == "densenet":
        model.add(layers.Dense(5, activation='sigmoid'))
        model.compile(
        loss='categorical_crossentropy',
        metrics=['acc']
        )
        
    if model_type == "resnet":
        model.add(layers.Dense(5, activation='sigmoid'))
        model.compile(
        loss='categorical_crossentropy',
        metrics=['acc']
        )
    
    print(model.summary())
    return model

def run_model(model, train_generator, validation_generator, metric_object, BATCH_SIZE):
#     with strategy.scope():
    
    train_steps_per_epoch = int(x_train.shape[0]*0.8) # train / validation split for train
    validation_steps_per_epoch = int(x_train.shape[0]*0.2) # train / validation split for validation
    
    history = model.fit(
        x=train_generator,
        steps_per_epoch=train_steps_per_epoch // BATCH_SIZE,
        epochs=15,
        validation_data=validation_generator,
#         use_multiprocessing=True,
#         workers=200,
        validation_steps = validation_steps_per_epoch // BATCH_SIZE,
        callbacks=[metric_object]
    )
    
    #scores = model.evaluate(appropriate_x_test, y_test)
    #print('Test loss:', scores[0])
    #print('Test accuracy:', scores[1])

In [ ]:
logging.basicConfig(filename='results.log', level=logging.INFO)

In [ ]:
x_train = images
y_train = multi_labels

prep_x_train_size_224 = np.array([preprocess_image_ben_grams(i, 224, 224) for i in x_train])
#prep_x_train_size_456 = np.array([preprocess_image_ben_grams(i, 456, 456) for i in x_train])

# DenseNet-121

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224

densenet = DenseNet121(
        weights='data/densenet_weights/DenseNet-BC-121-32-no-top.h5',
        include_top=False,
        input_shape=(IMG_WIDTH,IMG_HEIGHT,3)
    )

model = build_model(densenet, "densenet")

In [ ]:
BATCH_SIZE=32
train_generator = data_generator(IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, prep_x_train_size_224, train_valid="train")
val_generator = data_generator(IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, prep_x_train_size_224, train_valid="valid")

In [ ]:
# with strategy.scope():
#appropriate_x_test = np.array([preprocess_image_ben_grams(i, IMG_WIDTH, IMG_HEIGHT) for i in x_test])
logging.info("LOGS OF DENSENET-121 ARE STARTED")
kappa_metrics = Metrics()
MODEL_NAME = "DENSENET-121"
run_model(model, train_generator, val_generator, kappa_metrics, BATCH_SIZE)
logging.info("LOGS OF DENSENET-121 ARE FINISHED \n")

In [ ]:
# Visualize results
history_df = pd.DataFrame(model.history.history)
history_df[['loss', 'val_loss']].plot(figsize=(12,5))
plt.title("Loss (Categorical Crossentropy) for DenseNet-121", fontsize=16, weight='bold')
plt.xlabel("Epoch")
plt.ylabel("Loss (Categorical Crossentropy)")
history_df[['acc', 'val_acc']].plot(figsize=(12,5))
plt.title("Accuracy", fontsize=16, weight='bold')
plt.xlabel("Epoch")
plt.ylabel("% Accuracy")

# ResNet50
notice we do not need to generate new train and validation generators, because ResNet-50 uses the same image size as DenseNet-121

In [16]:
IMG_WIDTH, IMG_HEIGHT = 224, 224

resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT, 3))
model = build_model(resnet, "resnet")

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [ ]:
logging.info("LOGS OF RESNET-50 ARE STARTED")
kappa_metrics = Metrics()
MODEL_NAME = "RESNET-50"
run_model(model, train_generator, val_generator, kappa_metrics, BATCH_SIZE)
logging.info("LOGS OF RESNET-50 ARE FINISHED")

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousRandomSeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

...

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Exec

In [ ]:
# Visualize results
history_df = pd.DataFrame(model.history.history)
history_df[['loss', 'val_loss']].plot(figsize=(12,5))
plt.title("Loss (Categorical Crossentropy) for ResNet-50", fontsize=16, weight='bold')
plt.xlabel("Epoch")
plt.ylabel("Loss (Categorical Crossentropy)")
history_df[['acc', 'val_acc']].plot(figsize=(12,5))
plt.title("Accuracy", fontsize=16, weight='bold')
plt.xlabel("Epoch")
plt.ylabel("% Accuracy")

# EfficientNetB5

In [26]:
# IMG_WIDTH, IMG_HEIGHT = 456, 456

# effnet = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT, 3))
# model = build_model(effnet, "effnet")

Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /

In [28]:
# BATCH_SIZE=32
# train_generator = data_generator(IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, prep_x_train_size_456, train_valid="train")
# val_generator = data_generator(IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE, prep_x_train_size_456, train_valid="valid")

In [ ]:
# logging.info("LOGS OF EfficientNet-B5 ARE STARTED")
# kappa_metrics = Metrics()
# MODEL_NAME = "EfficientNet-B5"
# run_model(model, train_generator, val_generator, kappa_metrics, BATCH_SIZE)
# logging.info("LOGS OF EfficientNet-B5 ARE FINISHED")